* see: https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# two possible vector store
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

# removed OpenAI, using HF
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import hub

# removed OpenAI, using OCI GenAI
import oci

# oci_llm is in a local file
from oci_llm import OCIGenAILLM

from langchain.schema.runnable import RunnablePassthrough

# private configs
from config_private import (COMPARTMENT_OCID)

In [2]:
# to enable some debugging
DEBUG = False

In [3]:
# read OCI config to connect to OCI with API key
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file("~/.oci/config", CONFIG_PROFILE)

# OCI GenAI endpoint (for now Chicago)
ENDPOINT = "https://generativeai.aiservice.us-chicago-1.oci.oraclecloud.com"

# check the config to access to api keys
if DEBUG:
    print(config)

#### Loading the document

In [21]:
# BLOG_POST = "https://python.langchain.com/docs/get_started/introduction"
BOOK = "./oracle-database-23c-new-features-guide.pdf"
loader = PyPDFLoader(BOOK)

data = loader.load()

#### Splitting the document in chunks

In [22]:
CHUNK_SIZE = 1024

text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, 
                                               chunk_overlap=100)

splits = text_splitter.split_documents(data)

In [23]:
print(f"We have {len(splits)} splits...")

We have 237 splits...


In [24]:
# have a look at a single split
if DEBUG:
    print(splits[2])

#### Embeddings and Vectore Store

In [25]:
# We have substituted OpenAI with HF
EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}


hf = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_NAME, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# using Chroma or FAISS as Vector store
# vectorstore = Chroma.from_documents(documents=splits,
#                                    embedding=hf)
vectorstore = FAISS.from_documents(documents=splits, embedding=hf)

retriever = vectorstore.as_retriever()

#### Define the prompt structure

In [26]:
rag_prompt = hub.pull("rlm/rag-prompt")

#### Define the LLM

In [27]:
# compartment OCID from config_private.py

llm = OCIGenAILLM(
    temperature=1,
    max_tokens=1000,
    config=config,
    compartment_id=COMPARTMENT_OCID,
    endpoint=ENDPOINT,
    debug=DEBUG,
)

#### Define the (Lang)Chain

In [28]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | rag_prompt | llm

#### Process the question

In [29]:
# a list of possible questions
QUESTION1 = "What is the best architecture for an LLM?"
QUESTION2 = "What is LangChain?"
QUESTION3 = "Make a list of database 23c innovations in AI"
QUESTION4 = "Make a list of new features in Oracle database 23c"

In [30]:
%%time

# the question
QUESTION = QUESTION4

response = rag_chain.invoke(QUESTION)

print("The response:")
print(response)
print()

The response:
 Some of the new features in Oracle Database 23c include:

1. Add Verified SQL Plan Baseline
2. CMAN Diagnostics and Logging Enhancements
3. DBMS_DICTIONARY_CHECK PL/SQL Package
4. Estimate the Space Saved with Deduplication
5. Extent-Based Scrubbing
6. High Availability Diagnosability Using the DBMS_SCHEDULER Package
7. In-Memory Advisor
8. Oracle Call Interface (OCI) APIs to Enable Client-Side Tracing
9. Rename LOB Segment
10. Selective In-Memory Columns
11. Installation, Upgrade, and Patching
12. Upgrade
13. AutoUpgrade Release Update (RU) Upgrades
14. AutoUpgrade Sets Parallelism Based on System Resources
15. AutoUpgrade Supports Upgrades with Keystore Access to Databases Using TDE
16. AutoUpgrade Unplug-Plugin Upgrades to Different Systems
17. REST APIs for AutoUpgrade
18. xi

CPU times: user 76.1 ms, sys: 24.6 ms, total: 101 ms
Wall time: 9.85 s


#### Explore the vectore store

In [31]:
# Retrieve relevant splits for any question using similarity search.

# This is simply "top K" retrieval where we select documents based on embedding similarity to the query.

TOP_K = 5

docs = vectorstore.similarity_search(QUESTION, k=TOP_K)

len(docs)

5

In [32]:
for doc in docs:
    print(doc.page_content)
    print()

Oracle® Database
Oracle Database New Features
Release 23c
F48428-15
October 2023

Oracle Database Oracle Database New Features, Release 23c
F48428-15
Copyright © 2022, 2023, Oracle and/or its affiliates.
This software and related documentation are provided under a license agreement containing restrictions on
use and disclosure and are protected by intellectual property laws. Except as expressly permitted in your
license agreement or allowed by law, you may not use, copy, reproduce, translate, broadcast, modify, license,
transmit, distribute, exhibit, perform, publish, or display any part, in any form, or by any means. Reverse
engineering, disassembly, or decompilation of this software, unless required by law for interoperability, is
prohibited.
The information contained herein is subject to change without notice and is not warranted to be error-free. If
you find any errors, please report them to us in writing.
If this is software, software documentation, data (as defined in the Federal